# `multifunctional`

`multifunctional` is a library for handling multifunctional activities in the Brightway LCA software framework.

In [ ]:
import multifunctional as mf
import bw2data as bd
import bw2io as bi
import bw2calc as bc

In [ ]:
bi.remote.install_project(
    'ecoinvent-3.10-biosphere', 
    project_name="multifunctional demo", 
    overwrite_existing=True
)

In [ ]:
bd.projects.set_current("multifunctional demo")

In [ ]:
co2 = bd.get_node(
    name='Carbon dioxide, fossil',
    categories=('air', 'non-urban air or from high stacks'),
)

In [ ]:
mf_db = mf.MultifunctionalDatabase("emojis FTW")
mf_db.register(default_allocation="price")

In [ ]:
mf_data = {
    ("emojis FTW", "😼"): {
        "type": "product",
        "name": "meow",
        "unit": "kg",
    },
    ("emojis FTW", "🐶"): {
        "type": "product",
        "name": "woof",
        "unit": "kg",
    },
    ("emojis FTW", "1"): {
        "name": "process - 1",
        "location": "somewhere",
        "exchanges": [
            {
                "functional": True,
                "type": "production",
                "input": ("emojis FTW", "😼"),
                "amount": 4,
                "properties": {
                    "price": 7,
                    "mass": 6,
                },
            },
            {
                "functional": True,
                "type": "production",
                "input": ("emojis FTW", "🐶"),
                "amount": 6,
                "properties": {
                    "price": 12,
                    "mass": 4,
                },
            },
            {
                "functional": False,
                "type": "biosphere",
                "input": co2.key,
                "amount": 100
            }
        ],
    }
}

In [ ]:
mf_db.write(mf_data)

In [ ]:
for node in mf_db:
    print(node, node.id)
    for edge in node.edges():
        print("\t", edge)

In [ ]:
IPCC = (
    'IPCC 2021',
    'climate change',
    'global temperature change potential (GTP100)'
)

In [ ]:
lca = bc.LCA({bd.get_node(name="meow"): 3.141}, IPCC)
lca.lci()
lca.lcia()
lca.score

In [ ]:
mf_db.metadata["default_allocation"] = "mass"
mf_db.process()

In [ ]:
lca = bc.LCA({bd.get_node(name="meow"): 3.141}, IPCC)
lca.lci()
lca.lcia()
lca.score

# Custom allocation based on new properties

You need to tell the system to create a function for each new property

In [ ]:
mf.allocation_strategies['happiness'] = mf.property_allocation(property_label='😊')

In [ ]:
happy = {
    ("emojis FTW", "😼"): {
        "type": "product",
        "name": "meow",
        "unit": "kg",
    },
    ("emojis FTW", "🐶"): {
        "type": "product",
        "name": "woof",
        "unit": "kg",
    },
    ("emojis FTW", "1"): {
        "name": "process - 1",
        "location": "somewhere",
        "exchanges": [
            {
                "functional": True,
                "type": "production",
                "input": ("emojis FTW", "😼"),
                "amount": 4,
                "properties": {
                    "price": 7,
                    "mass": 6,
                    "😊": 10,
                },
            },
            {
                "functional": True,
                "type": "production",
                "input": ("emojis FTW", "🐶"),
                "amount": 6,
                "properties": {
                    "price": 12,
                    "mass": 4,
                },
            },
            {
                "functional": False,
                "type": "biosphere",
                "input": co2.key,
                "amount": 100
            }
        ],
    }
}
mf_db.write(happy)

We can check to make sure that the properties we want are available:

In [ ]:
mf.list_available_properties("emojis FTW")

In [ ]:
happy_dog = {
    ("emojis FTW", "😼"): {
        "type": "product",
        "name": "meow",
        "unit": "kg",
    },
    ("emojis FTW", "🐶"): {
        "type": "product",
        "name": "woof",
        "unit": "kg",
    },
    ("emojis FTW", "1"): {
        "name": "process - 1",
        "location": "somewhere",
        "exchanges": [
            {
                "functional": True,
                "type": "production",
                "input": ("emojis FTW", "😼"),
                "amount": 4,
                "properties": {
                    "price": 7,
                    "mass": 6,
                    "😊": 2,
                },
            },
            {
                "functional": True,
                "type": "production",
                "input": ("emojis FTW", "🐶"),
                "amount": 6,
                "properties": {
                    "price": 12,
                    "mass": 4,
                    "😊": 50,
                },
            },
            {
                "functional": False,
                "type": "biosphere",
                "input": co2.key,
                "amount": 100
            }
        ],
    }
}
mf_db.write(happy_dog)

In [ ]:
mf_db.metadata["default_allocation"] = "happiness"
mf_db.process()

In [ ]:
lca = bc.LCA({bd.get_node(name="meow"): 3.141}, IPCC)
lca.lci()
lca.lcia()
lca.score

# Writing custom allocation functions

In [ ]:
from functools import partial

def allocation_factor(edge_data: dict, node: mf.MaybeMultifunctionalProcess) -> float:
    """Nonsensical allocation factor generation"""
    if edge_data['input'] == ('emojis FTW', '😼'):
        return 7
    else:
        return 17

mf.allocation_strategies['silly'] = partial(
    mf.generic_allocation,
    func=allocation_factor,
    strategy_label="something silly"
)

In [ ]:
mf_db.metadata["default_allocation"] = "silly"
mf_db.process()

In [ ]:
lca = bc.LCA({bd.get_node(name="meow"): 3.141}, IPCC)
lca.lci()
lca.lcia()
lca.score